# Installing libraries
This part helps install libraries that are not available OOTB.

In [ ]:
import sys
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install numpy


# Imports

Here we import all libraries we'll use and install `matplotlib` and `numby` modules needed for further work.

In [ ]:
import random
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import csv
import math

# Loading data

In [ ]:
X = []
Y = []
with open('board_games.csv', 'r') as csvfile:
    spamreader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
    for row in spamreader:
        X.append(int(row['owned']))
        Y.append(int(row['wishlist']))

# Visualize the data

In [ ]:
FONT_SIZE = 16
plt.rc('font', size=FONT_SIZE)
plt.rc('axes', titlesize=FONT_SIZE)
plt.rcParams['figure.figsize'] = [20, 10]
plt.scatter(X,Y,marker="o")
plt.xlabel("OWNED")
plt.ylabel("WISHLISTED")
plt.show()

# Generating population
We're looking for optimal parameters _a_ and _b_ for the $f(x) = a\cdot x + b$ equation that is used for linear regression of the data.

Our individual will take a a binary form of which first half codes the $a$ parameter and second half codes the $b$ parameter.

Individual: \[binary_parameter_a | binary_parameter_b\], e.g. \[0 0 0 1 | 1 0 1 0\]

In [ ]:
def generate_binary_phenotype(size: int):
    return [random.randrange(0, 2, 1) for i in range(size)]

def generate_population(population_size: int, phenotype_size: int, generator):
    return [generator(phenotype_size) for i in range(population_size)]

population = generate_population(100, 32, generate_binary_phenotype)

print(f"Population size: {len(population)}")
print(f"First individual: {str(population[0])}")

In [ ]:
def decode_individual(individual):
    half = int(len(individual)/2)
    a = int("".join(str(x) for x in individual[0:half]), 2)
    b = int("".join(str(x) for x in individual[half:]), 2)
    a = a/1000
    b = b/1000
    return a, b

def fitness_function(individual):
    a, b = decode_individual(individual)
    fitness = 0
    for i in range(len(X)):
        approximation = a*X[i]+b
        fitness = fitness + ((Y[i] - approximation)**2)
    return individual, fitness/len(X)

first_individual = population[0]
first_individual, fitness_value = fitness_function(first_individual)
test_a, test_b = decode_individual(first_individual)
print(f"Test individual: {str(first_individual)} codes {test_a}x + {test_b} function")
print(f"Test individual's fitness value: {fitness_value}")

# Let's see what randomness has provided

In [ ]:
individual_fitness_pairs = list(map(fitness_function, population))
best = min(individual_fitness_pairs, key=lambda item: item[1])
worst = max(individual_fitness_pairs, key=lambda item: item[1])

a_best, b_best = decode_individual(best[0])
a_worst, b_worst = decode_individual(worst[0])

plt.rcParams['figure.figsize'] = [20, 10]

figure, axis = plt.subplots(ncols=2)

x_best = np.arange(0.0, 175000.0, 1.0)
y_best = np.add(np.multiply(x_best,a_best), b_best)

x_worst = np.arange(0.0, 175000.0, 1.0)
y_worst = np.add(np.multiply(x_worst,a_worst), b_worst)

axis[0].scatter(X,Y,marker="o")
axis[1].scatter(X,Y,marker="o")
axis[0].plot(x_best, y_best)
axis[1].plot(x_worst, y_worst)

plt.show()

# Adding artificial intelligence

## Mutation

In [ ]:
def mutation(individual, initial_mutation_rate):
    mutation_rate = 1/len(individual) if initial_mutation_rate < 0 else initial_mutation_rate
    new_phenotype = []
    for value in individual:
        new_phenotype.append( abs(value - 1) if random.random() < mutation_rate else value)
    return new_phenotype

print ("Original phenotype: " +  str(worst[0]))
mutant = mutation(worst[0], 0.5)
print ("Mutated phenotype : " +  str(mutant))

## Recombination

In [ ]:
def recombination(individual_1, individual_2):
    crossover_point = random.randrange(1, len(individual_1), 1)
    child_1 = individual_1[:crossover_point] + individual_2[crossover_point:]
    child_2 = individual_2[:crossover_point] + individual_1[crossover_point:]
    return child_1, child_2

test_1 = [1, 1, 1, 1, 1, 1, 1, 1]
test_2 = [0, 0, 0, 0, 0, 0, 0, 0]
child_1, child_2 = recombination(test_1, test_2)
print(f"Children: {str(child_1)} & {str(child_2)}")

## Selection

In [ ]:
test = [('a', 0), ('b', 1), ('c', 2), ('d', 3), ('A', 100), ('B', 99), ('C', 101), ('D', 102) ]
def simple_selection(individual_fitness_pairs):
    first = individual_fitness_pairs[random.randrange(0, len(individual_fitness_pairs), 1)]
    individual_fitness_pairs.remove(first)
    second = individual_fitness_pairs[random.randrange(0, len(individual_fitness_pairs), 1)]
    individual_fitness_pairs.remove(second)
    return first[0], second[0], individual_fitness_pairs

f1, s1, ind = simple_selection(test)
f2, s2, ind = simple_selection(ind)
f3, s3, ind = simple_selection(ind)

print(f'First pair : ({f1}, {s1})')
print(f'Second pair: ({f2}, {s2})')
print(f'Third pair : ({f3}, {s3})')

In [ ]:
test = [('a', 0), ('b', 1), ('c', 2), ('d', 3), ('A', 100), ('B', 99), ('C', 101), ('D', 102) ]
def monte_carlo_selection(individual_fitness_pairs):
    rulette_wheel = []
    temp = list(map(lambda x : (x[0], math.ceil(10000/(x[1] if x[1] != 0 else 1))), individual_fitness_pairs))
    for index in range(len(temp)):
        rulette_wheel = rulette_wheel + temp[index][1]*[index]
    rand = random.randrange(0, len(rulette_wheel), 1)
    first = individual_fitness_pairs[rulette_wheel[rand]]
    rulette_wheel = [index for index in rulette_wheel if index != rulette_wheel[rand]]
    rand = random.randrange(0, len(rulette_wheel), 1)
    second = individual_fitness_pairs[rulette_wheel[rand]]
    individual_fitness_pairs.remove(first)
    individual_fitness_pairs.remove(second)
    return first[0], second[0], individual_fitness_pairs
    
f1, s1, ind = monte_carlo_selection(test)
f2, s2, ind = monte_carlo_selection(ind)
f3, s3, ind = monte_carlo_selection(ind)

print(f'First pair : ({f1}, {s1})')
print(f'Second pair: ({f2}, {s2})')
print(f'Third pair : ({f3}, {s3})')

# Solution time
Now we'll combine all the functions together to find the result

In [ ]:
def breedResult(population, mutation_rate, elite_size, epochs, fitness_f, mutation_f, recombination_f, selection_f):
    pop_size = len(population)
    best_history = []
    current_population = population
    for epoch in range(epochs):
        individual_fitness_pairs = list(map(fitness_f, current_population))
        individual_fitness_pairs.sort(key=lambda x : x[1])
        best_history.append(individual_fitness_pairs[0][0])
        new_population = [individual[0] for individual in individual_fitness_pairs[:elite_size]]
        while len(new_population) < pop_size:
            first, second, individual_fitness_pairs = selection_f(individual_fitness_pairs)
            first, second = recombination_f(first, second)
            first = mutation_f(first, mutation_rate)
            second = mutation_f(second, mutation_rate)
            new_population.append(first)
            new_population.append(second)
        current_population = new_population
    return best_history

results = breedResult(population, -1, 10, 1000, fitness_function, mutation, recombination, monte_carlo_selection)

In [ ]:
a_best, b_best = decode_individual(results[-1])

plt.rcParams['figure.figsize'] = [20, 10]

figure, axis = plt.subplots(ncols=2)

x_best = np.arange(0.0, 175000.0, 1.0)
y_best = np.add(np.multiply(x_best,a_best), b_best)

x_worst = np.arange(0.0, 175000.0, 1.0)
for result in results:
    current = decode_individual(result)
    a_worst, b_worst = decode_individual(result)
    y_worst = np.add(np.multiply(x_worst,a_worst), b_worst)
    axis[1].plot(x_worst, y_worst)

axis[0].scatter(X,Y,marker="o")
axis[1].scatter(X,Y,marker="o")
axis[0].plot(x_best, y_best)

plt.show()
print(f"Best solution {a_best}x + {b_best}")

# Proving I'm right - square regression
To show that the line was selected the best way Genetic Algorithm could do it with the given limitations, we'll do the same exercise for a more complex equation: $f(x) = a\cdot x^2 + b\cdot x + c$.

In [ ]:
def decode_individual_sq(individual):
    third = int(len(individual)/3)
    a = int("".join(str(x) for x in individual[0:third]), 2)
    b = int("".join(str(x) for x in individual[third:(third + third)]), 2)
    c = int("".join(str(x) for x in individual[(third + third):]), 2)
    a = a/1000
    b = b/1000
    c = c/1000
    return a, b, c

def fitness_function_sq(individual):
    a, b, c = decode_individual_sq(individual)
    fitness = 0
    for i in range(len(X)):
        fitness = fitness + (Y[i] - (a*X[i]**2+b*X[i]+c))**2
    return individual, fitness/len(X)

population_sq = generate_population(200, 48, generate_binary_phenotype)

results_sq = breedResult(population_sq, -1, 10, 100, fitness_function_sq, mutation, recombination, monte_carlo_selection)

a_best_sq, b_best_sq, c_best_sq = decode_individual_sq(results_sq[-1])
a_first_sq, b_first_sq, c_first_sq = decode_individual_sq(results_sq[1])

plt.rcParams['figure.figsize'] = [20, 10]

x_best_sq = np.arange(0.0, 175000.0, 1.0)
y_best_sq = np.add(np.multiply(np.multiply(x_best_sq,x_best_sq),a_best_sq), np.add(np.multiply(x_best_sq,b_best_sq), c_best_sq))
y_first_sq = np.add(np.multiply(np.multiply(x_best_sq,x_best_sq),a_first_sq), np.add(np.multiply(x_best_sq,b_first_sq), c_first_sq))

plt.scatter(X,Y,marker="o"),
plt.plot(x_best_sq, y_best_sq)
# plt.plot(x_best, y_first)
plt.xlabel("How many people own the game")
plt.ylabel("How many times the game was played")
plt.show()
print(f"Best solution {a_best_sq}x^2 + {b_best_sq}x + {c_best_sq}")

# And now for something completely different
We have a distance matrix that can be understand as:

|       | A | B | C | D | E |
|-------|---|---|---|---|---|
| **A** | 0 | 2 | 3 | 4 | 5 |
| **B** | 2 | 0 | 1 | 2 | 3 |
| **C** | 1 | 2 | 0 | 3 | 1 |
| **D** | 4 | 3 | 2 | 0 | 1 |
| **E** | 1 | 5 | 2 | 3 | 0 |

Let's find the shortest route!

In [ ]:
distance_matrix = [
    [0, 2, 3, 4, 5],
    [2, 0, 1, 2, 3],
    [1, 2, 0, 3, 1],
    [4, 3, 2, 0, 1],
    [1, 5, 2, 3, 0]
]

## Population & Fitness
We need to change our view on generating a phenotype and calculating its fitness.

In [ ]:
def generate_phenotype_map(size: int):
    phenotype = list(range(0, size))
    random.shuffle(phenotype)
    return phenotype

def fitness_function_map(individual):
    fitness = 0
    for i in range(1, len(individual)):
        fitness = fitness + distance_matrix[individual[i-1]][individual[i]]
    return individual, fitness

population_map = generate_population(10, 5, generate_phenotype_map)

print(f'Fitness of {population_map[0]} is {fitness_function_map(population_map[0])[1]}')

## Mutation
Instead of changing value to the opposite we exchange places of the indexes in the list.

In [ ]:
def mutation_map(individual, initial_mutation_rate):
    mutation_rate = 1/len(individual) if initial_mutation_rate < 0 else initial_mutation_rate
    new_phenotype = list(individual)
    for i in range(len(new_phenotype)):
        if random.random() < mutation_rate:
            index = random.choice(range(0, len(new_phenotype)))
            new_phenotype[i], new_phenotype[index] = new_phenotype[index], new_phenotype[i]
    return new_phenotype

test = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
print(mutation_map(test, -1))

## Recombination
This one's tricky. We take a part of one individual and then supplement the second part with missing indexes from the second 'parent' in order, in which they appear in it.

In [ ]:
def recombination_map(individual_1, individual_2):
    def add_missing(child, parent):
        new_child = list(child)
        for index in parent:
            if index not in new_child:
                new_child.append(index)
        return new_child
    crossover_point = random.randrange(1, len(individual_1), 1)
    child_1 = add_missing(individual_2[:crossover_point], individual_1)
    child_2 = add_missing(individual_1[:crossover_point], individual_2)
    return child_1, child_2

test_1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
test_2 = [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
child_1, child_2 = recombination_map(test_1, test_2)
print(f"Children: {str(child_1)} & {str(child_2)}")

# Let's solve this!
We just put our new functions in the old loop and calculate the answer

Psssst... we're expecting \[3, 4, 0, 1, 2\] as a result ;)

In [ ]:
results_map = breedResult(population_map, -1, 2, 10, fitness_function_map, mutation_map, recombination_map, monte_carlo_selection)
print(f'The result is {results_map[-1]} with fitness of {fitness_function_map(results_map[-1])[1]}')

We can easily calculate that for **population** of *10* in *10* **epochs** we have generated 100 individuals. If we did the same with brute force it'd be *5!* or *120*. So we solved it intelligently.